In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install openai-whisper
pip install python-dotenv
pip install pydub

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import whisper
import math
from pydub import AudioSegment
import librosa
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import soundfile as sf
os.chdir('/content/drive/MyDrive/Colab Notebooks/kakaotech/피칭/')
load_dotenv()
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# model = whisper.load_model("large")
def transcribe_audio(file_path): # STT 처리
    global model
    result = model.transcribe(file_path, fp16=False)
    return result["text"]

def generate_tts_audio(script): # TTS
    num = math.ceil(len(script)/4000)
    if math.ceil(num) == 1:
      response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input = script
      )
      response.stream_to_file("TTS.mp3")

    else:
      for i in range(num):
        response = client.audio.speech.create(
          model="tts-1",
          voice="alloy",
          input = script[4000*(i):4000*(i+1)]
        )
        response.stream_to_file("TTS" + str(i) + ".mp3")

      audio1 = AudioSegment.from_mp3("TTS0.mp3")
      audio2 = AudioSegment.from_mp3("TTS1.mp3")
      combined_audio = audio1 + audio2
      combined_audio.export("TTS.mp3", format="mp3")


def compare_audio_similarity(file1, file2): # 음성 유사도 비교
    y1, sr1 = librosa.load(file1, sr=None)
    y2, sr2 = librosa.load(file2, sr=None)

    min_len = min(len(y1), len(y2))
    y1 = y1[:min_len]
    y2 = y2[:min_len]

    mfcc1 = librosa.feature.mfcc(y=y1, sr=sr1)
    mfcc2 = librosa.feature.mfcc(y=y2, sr=sr2)

    similarity = cosine_similarity(mfcc1.T, mfcc2.T).mean()
    return similarity

def calculate_text_accuracy(reference_text, transcribed_text): # 텍스트 유사도
    reference_words = reference_text.split()
    transcribed_words = transcribed_text.split()

    correct_words = sum(1 for ref, trans in zip(reference_words, transcribed_words) if ref == trans)
    accuracy = correct_words / max(len(reference_words), len(transcribed_words))
    return accuracy

def calculate_presentation_score(audio_file_path, script_text): # 최종 점수 산출 함수
    transcribed_text = transcribe_audio(audio_file_path)
    print("Transcribed Text:", transcribed_text)

    text_accuracy = calculate_text_accuracy(script_text, transcribed_text)
    print("Text Accuracy:", text_accuracy)

    tts_file_path = "TTS.mp3"
    generate_tts_audio(script_text)

    audio_similarity = compare_audio_similarity(audio_file_path, tts_file_path)
    print("Audio Similarity:", audio_similarity)

    final_score = (text_accuracy * 0.5) + (audio_similarity * 0.5)
    return final_score

In [ ]:
audio_file_path = "attention is all you need.mp3"  # 발표자의 음성 파일 경로
script_text = "꼼꼼한 딥러닝 논문 리뷰와 코드 실습. 이번 시간에 리뷰할 논문은 현대 딥러닝 기반의 자연어처리 기술의 핵심 아키텍처가 되고 있는 트랜스포머입니다. 트랜스포머 논문의 원래 제목은 Attention is all you need입니다. 논문의 제목에서 알 수 있듯이 트랜스포머라는 아키텍처에는 이 Attention이라고 하는 것이 가장 메인 아이디어로서 사용이 된다는 걸 알 수 있습니다. 실제로 트랜스포머는 Attention이라는 메커니즘을 전적으로 활용하는 아키텍처입니다. 트랜스포머가 나오게 된 계기를 이해하기 위해서 딥러닝 기반의 기계 번역 발전 과정에 대해 확인해 보겠습니다. 2021년 기준으로 최신 자연어처리 쪽 고성능 모델들은 이런 트랜스포머 아키텍처를 기반으로 하고 있습니다. 최근까지 화제가 되었던 GPT와 BERT는 모두 이러한 트랜스포머의 아키텍처를 적절히 활용하여 좋은 성능을 내고 있습니다. 대표적으로 GPT는 트랜스포머의 디코드 아키텍처를 활용했고 BERT는 트랜스포머의 인코드 아키텍처를 활용했다는 점입니다. 자연어처리 테스크 중에서 가장 대표적이면서 중요한 테스크 중 하나는 기계 번역입니다. 실제로 기계 번역 기술의 발전 과정을 확인해 보시면 1986년도 즈음에 RNN이 제한되었고 그로부터 약 10년 정도가 지난 뒤에 LSTM이 등장하였습니다. 이러한 LSTM을 활용하면 다양한 시퀀스 정보를 모델링할 수 있는데요. 대표적으로 주가 예측, 주기함수 예측 등이 가능합니다. 이러한 LSTM을 활용해서 2014년도에는 딥러닝 기반 기술로 시퀀스 정보를 모델링할 수 있는데요. 시퀀스 시퀀스가 등장하였습니다. 시퀀스 시퀀스는 현대의 딥러닝 기술들이 다시 빠르게 나오기 시작한 시점인 2014년도에 이러한 LSTM을 활용해서 고정된 크기의 컨텍스트 벡터를 사용하는 방식으로 번역을 수행하는 방법을 제안하였습니다. 다만 이러한 시퀀스 시퀀스 모델이 나왔을 때의 시점만 하더라도 고정된 크기의 컨텍스트 벡터를 쓰고 있기 때문에 소스 문장을 전부 고정된 크기의 한 벡터에다가 압축을 할 필요가 있다는 점에서 성능적인 한계입니다. 이 시스템을 통해 LSTM을 활용한 것에 대한 성능적인 한계가 존재했습니다. 이후에 어텐션 메커니즘이 제한된 논문이 나오면서 이러한 시퀀스 시퀀스 모델에 어텐션 기법을 적용하여 성능을 더 끌어올릴 수가 있었고요. 이제 그 이후에 트랜스포머 논문에서는 그냥 RNN 자체를 사용할 필요가 없다는 아이디어로 오직 어텐션 기법에 의존하는 아키텍처를 설계했더니 성능이 훨씬 좋아지는 것을 보여주었습니다. 즉 이 트랜스포머를 기점으로 해서 더 이상 다양한 자연어처리 테스크에 대해서 RNN 기반의 아키텍처를 사용하지 않고 어텐션 메커니즘을 더욱더 많이 사용하게 되었습니다. 그래서 어텐션 메커니즘이 등장한 이후로부터는 입력 시퀀스 전체에서 정보를 추출하는 방향으로 연구 방향이 발전되어 왔다고 할 수 있습니다. 물론 이후에 나온 논문들 중에서도 RNN을 활용하는 아키텍처도 많이 존재하지만 전반적인 추세 자체는 어텐션 기법을 더욱더 활용하는 이런 트랜스포머의 아키텍처를 따르는 방식으로 다양한 고성능 모델들이 제한되고 있습니다. 그렇다면 기존에 제한되었던 시퀀스트 시퀀스는요. 시퀀스 모델에는 어떤 한계점이 존재할까요? 기존 시퀀스트 시퀀스 모델의 한계점이라고 한다면 이 컨텍스트 벡터 V에 소스 문장의 정보를 압축한다는 점입니다. 이때 병목 현상이 발생할 수 있기 때문에 성능 하락의 원인이 될 수 있는데요. 현재 예시를 확인해 보시면 대표적인 시퀀스트 시퀀스 모델을 활용한 기계 번역 예시라고 할 수 있습니다. 왼쪽에 있는 독일어 문장, 즉 각각의 단어들로 구성된 하나의 시퀀스가 들어왔을 때 이렇게 중간에서 하나의 고정된 크기의 컨텍스트 벡터로 바꾼 뒤에 다시 이러한 컨텍스트 벡터로부터 출력 문장을 만들어내는 것을 확인할 수 있습니다. 즉 한쪽의 시퀀스에서부터 다른 한쪽의 시퀀스를 만든다는 의미에서 시퀀스트 시퀀스 모델이라고 부를 수 있습니다. 결과적으로 이렇게 영어 출력 문장이 나오는 걸 확인할 수 있고요. 다만 이때 이러한 시퀀스트 시퀀스 아키텍처를 확인해 보시면 매번 단어가 입력될 때마다 히든 스테이트 값을 갱신하는 걸 확인할 수 있습니다. 이런 식으로 단어가 입력될 때마다 이전까지 입력되면 단어들에 대한 정보를 포함하고 있는 히든 스테이트 값을 받아서 매번 이런 식으로 히든 스테이트 값을 새롭게 갱신합니다. 즉 이런 식으로 각각의 단어가 차례대로 순서에 맞게 입력될 때마다 히든 스테이트 값이 갱신되어 이러한 히든 스테이트 값은 이전까지 입력되었던 단어들에 대한 정보를 갖고 있기 때문에 이렇게 마지막 단어가 들어왔을 때 그때의 히든 스테이트 값은 소스 문장 전체를 대표하는 하나의 컨텍스트 벡터로서 사용할 수가 있다는 것입니다. 그렇기 때문에 이렇게 마지막 단어가 들어왔을 때의 히든 스테이트 값을 하나의 컨텍스트 벡터로서 이 컨텍스트 벡터 안에는 앞에 등장했던 소스 문장에 대한 문맥적인 정보를 담고 있다고 가정하는 것입니다. 그렇기 때문에 이러한 컨텍스트 벡터로부터 출발해서 이렇게 출력을 수행하는 디코더 파트에서는 매번 출력 단어가 들어올 때마다 이러한 컨텍스트 벡터로부터 출발해서 마찬가지로 히든 스테이트를 만들어서 매번 출력을 내보냅니다. 이렇게 그 다음 단계에서는 이렇게 출력했던 단어가 다시 입력으로 들어와서 반복적으로 이전까지 출력했던 단어에 대한 정보를 가지고 있는 히든 스테이트와 같이 입력을 받아 새롭게 히든 스테이트를 갱신하는 걸 확인할 수 있습니다. 이런 식으로 디코더 파트에서는 매번 히든 스테이트 값을 갱신하면서 이렇게 히든 스테이트 값으로부터 출력 값이 엔드 오브 시퀀스가 나올 때까지 반복합니다. 그래서 엔드 오브 시퀀스가 나왔을 때 출력 문장 생성을 마치게 되고요. 이렇게 출력된 정보인 good evening이 나오는 걸 확인할 수 있습니다. 가장 기본적인 형태의 시퀀스 시퀀스 모델의 동작 원리입니다. 다만 확인해 보시면 이렇게 소스 문장을 대표하는 하나의 컨텍스트 벡터를 만들어야 한다는 점에서 이렇게 고정된 크기의 컨텍스트 벡터의 정보를 압축하려고 하면 이러한 입력 문장은 어떨 때는 짧기도 하고 어떨 때는 길기도 하기 때문에 그러한 다양한 경우의 수에 대해서 항상 소스 문장의 정보를 고정된 크기로 가지고 있는 것은 전체 성능에서 병목 현상의 원인이 될 수 있습니다. 그래서 이러한 문제를 조금이나마 완화하기 위한 아이디어로 이 고정된 크기의 컨텍스트 벡터를 매번 이 디코더의 RNN 셀에서 참고하도록 만들어서 조금 더 성능을 개선할 수 있습니다. 이렇게 하게 되면 이 컨텍스트 벡터에 대한 정보가 이 디코더 파트의 RNN 셀을 거침에 따라서 정보가 손실되는 정도를 더 줄일 수 있기 때문에 출력되는 문장이 길어진다고 하더라도 각각의 출력되는 단어에 이러한 컨텍스트 벡터에 대한 정보를 다시 한번 넣어 줄 수 있어서 성능이 기존보다 조금 더 향상될 수 있습니다. 그래서 이러한 컨텍스트 벡터에 대한 정보를 다시 한번 넣어 줄 수 있어서 성능이 기존보다 조금 더 향상될 수 있습니다. 다만 이런 식으로 접근한다고 하더라도 여전히 이 소스 문장을 하나의 벡터에 압축해야 된다는 점은 동일하기 때문에 병목현상은 여전히 발생합니다. 즉 현재의 문제 상황이라고 한다면 하나의 문맥 벡터, 즉 컨텍스트 벡터가 소스 문장의 모든 정보를 가지고 있어야 하기 때문에 성능이 저하될 수 있다는 것입니다. 그렇다면 디코더 파트에서는 하나의 문맥 벡터에 대한 정보만 가지고 있는 게 아니라 출력 단어를 만들 때마다 매번 소스 문장에서의 출력값들 전부를 입력으로 받으면 어떨까요? 라는 아이디어가 나올 수 있는 거죠. 최신 GPU는 많은 메모리와 그리고 빠른 병렬 처리를 지원하기 때문에 소스 문장의 시퀀스 길이가 길다고 하더라도 그러한 소스 문장을 구성하는 각각의 단어에 대한 출력값들 전부를 특정 행렬에다가 기록해 놓았다가 소스 문장에 대한 전반적인 내용들을 매번 출력할 때마다 반영할 수 있기 때문에 성능이 좋아질 것을 기대할 수 있습니다. 다시 말해 하나의 고정된 크기의 컨텍스트 벡터에 담지 말고 그냥 소스 문장에서 나왔던 출력값들 전부를 매번 입력으로 받아서 일련의 처리 과정을 거쳐서 출력 단어를 만들도록 하면 성능이 더 좋아질 수 있다는 겁니다. 지금 보이는 아키텍처가 바로 시퀀스 시퀀스에 어텐션 메커니즘을 적용한 아키텍처인데요. 이렇게 어텐션 메커니즘을 적용해서 인코더 파트의 모든 출력을 참고하도록 만들 수가 있습니다. 실제로 파이톨치와 같은 프레임워크에서는 단순히 RNN이나 LSTM 같은 걸 사용하도록 만들면 이렇게 매번 전체 시퀀스 길이에 맞는 아웃풋 값들이 따로 출력 값들이 나오게 되는데요. 이제 그걸 그대로 이용해서 실제로 어텐션 메커니즘을 간단하게 구현할 수도 있습니다. 전반적인 내용을 확인해 보시면 이렇게 매번 단어가 출력돼서 히든스테이트가 나올 때마다 그냥 이 값들을 전부 다 출력 값으로써 그냥 별도의 배열에다가 다 기록해 놓습니다. 그래서 이런 식으로 각각의 단어를 거치면서 갱신되는 히든스테이트 값들을 매번 다 가지고 있는 거예요. 이렇게 해 줌으로써 이렇게 매 단어가 들어왔을 때에 히든스테이트 값을 전부 다 출력할 수 있습니다. 그리고 이 값들을 어떻게든 참고해서 이렇게 출력 단어가 매번 생성될 때마다 이러한 소스 문장 전체를 반영하겠다라는 아이디어라고 보시면 되겠습니다. 실제로는 이렇게 디코더 파트에서 매번 히든스테이트를 갱신하게 되는데 이때 현재 단계에서 히든스테이트 값을 만든다고 하면 바로 이전에 히든스테이트 값을 이용해서 이 출력 단의 히든스테이트 값과 이렇게 소스 문장 단의 히든스테이트 값을 서로 묶어서 별도의 행렬 곱을 수행해서 각각 에너지 값을 만들어 냅니다. 이때 그 에너지 값은 내가 현재 어떠한 단어를 출력하기 위해서 소스 문장에서 어떤 단어에 초점을 둘 필요가 있는지를 수치화해서 표현한 값입니다. 그래서 그러한 에너지 값에 소프트 맥스를 취해서 확률 값을"
score = calculate_presentation_score(audio_file_path, script_text)
print("Final Presentation Score:", score)

In [ ]:
audio_file_path = "도비 발표.m4a"
script_text = transcribe_audio(audio_file_path)

In [ ]:
script_text

' 발표 시작하겠습니다. 안녕하세요. 저는 이번에 인공지능과 파이썬으로 금융 데이터 분석해보기 with 자연어 처리로 발표하게 될 김대현이라고 합니다. 일단 제 소개를 간단하게 드리자면, 저는 지금 모드웨어 연구소라는 곳에서 비전 랭키지 쪽으로 모드웨어 연구를 하고 있고요. 또한 카카오테크 부트캡에서 생성형 인공지능 과정을 수강하고 있습니다. 최근에 고파콘이라고 고언어 관련해서 열리는 큰 행사가 있는데, 거기에 준비위원회로 참가를 했고, 이제 또 JRC라는 로봇회사에서 로보틱스 엔지니어로 근무한 경력이 있습니다. 일단 크게 4가지 주제를 다뤄볼 거예요. TF, IDF, N-gram, Covert, DinoSync 메서드랑 Positive Index Reality, 즉 이제 신뢰지수에 대해서 한번 얘기를 해보려고 합니다. 일단은 이 프로젝트를 먼저 시작하게 된 계기가 바로 이거였어요. 일반적으로 우리가 주식을 할 때 처음에 화면을 보면 어떠한 걸 봐야 될지 모르겠는 거예요. 그래서 약간 복잡한 리서치 화면 및 뉴스 커뮤니티에 통한 투자 결정에 어려움이 존재해서 이 프로젝트를 진행하게 되었고, 또한 이제 일단은 한국 투자증권 기준으로 보면은 카테고리가 일단은 종목이 2400개가 있어요. 근데 이거를 10개의 섹터로만 구분을 해서 의사결정에 대해서 어려운 점이 있었습니다. 그래서 이 프로젝트에 진행하게 되었고, 요약해드리자면 이제 우리가 이걸 왜 하게 되었는지, 중요정보에 쉽게 접근하고, 이제 정확한 투자 도구에 대해서, 그래서 이 프로젝트에 진행하게 되었고, 요약해드리자면 이제 우리가 이걸 왜 하게 되었는지, 중요정보에 쉽게 접근하고, 이제 정확한 투자 도구에 대해서, 정확한 투자 동향을 파악하면서 이제 유망한 분야에 탐색하고, 또한 이제 여론과 어느 정도 상관관계 분석이 있는지 그걸 파악하면서 개발해보려고 개발했습니다. 일단은 개발 설계 및 연구 방법에 대해서 설명을 드리면은, 일단은 모델을 선정이랑 이제 어떻게 프로젝트를 진행할지 플래닝을 했고요. 건 다음에 

In [ ]:
generate_tts_audio(script_text)

<ipython-input-16-d4b89a8e9357>:41: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("TTS" + str(i) + ".mp3")


In [ ]:
score = calculate_presentation_score(audio_file_path, "TTS.mp3") #
print("Final Presentation Score:", score)

Transcribed Text:  발표 시작하겠습니다. 안녕하세요. 저는 이번에 인공지능과 파이썬으로 금융 데이터 분석해보기 with 자연어 처리로 발표하게 될 김대현이라고 합니다. 일단 제 소개를 간단하게 드리자면, 저는 지금 모드웨어 연구소라는 곳에서 비전 랭키지 쪽으로 모드웨어 연구를 하고 있고요. 또한 카카오테크 부트캡에서 생성형 인공지능 과정을 수강하고 있습니다. 최근에 고파콘이라고 고언어 관련해서 열리는 큰 행사가 있는데, 거기에 준비위원회로 참가를 했고, 이제 또 JRC라는 로봇회사에서 로보틱스 엔지니어로 근무한 경력이 있습니다. 일단 크게 4가지 주제를 다뤄볼 거예요. TF, IDF, N-gram, Covert, DinoSync 메서드랑 Positive Index Reality, 즉 이제 신뢰지수에 대해서 한번 얘기를 해보려고 합니다. 일단은 이 프로젝트를 먼저 시작하게 된 계기가 바로 이거였어요. 일반적으로 우리가 주식을 할 때 처음에 화면을 보면 어떠한 걸 봐야 될지 모르겠는 거예요. 그래서 약간 복잡한 리서치 화면 및 뉴스 커뮤니티에 통한 투자 결정에 어려움이 존재해서 이 프로젝트를 진행하게 되었고, 또한 이제 일단은 한국 투자증권 기준으로 보면은 카테고리가 일단은 종목이 2400개가 있어요. 근데 이거를 10개의 섹터로만 구분을 해서 의사결정에 대해서 어려운 점이 있었습니다. 그래서 이 프로젝트에 진행하게 되었고, 요약해드리자면 이제 우리가 이걸 왜 하게 되었는지, 중요정보에 쉽게 접근하고, 이제 정확한 투자 도구에 대해서, 그래서 이 프로젝트에 진행하게 되었고, 요약해드리자면 이제 우리가 이걸 왜 하게 되었는지, 중요정보에 쉽게 접근하고, 이제 정확한 투자 도구에 대해서, 정확한 투자 동향을 파악하면서 이제 유망한 분야에 탐색하고, 또한 이제 여론과 어느 정도 상관관계 분석이 있는지 그걸 파악하면서 개발해보려고 개발했습니다. 일단은 개발 설계 및 연구 방법에 대해서 설명을 드리면은, 일단은 모델을 선정이랑 이제 어떻게 프로젝트를 진행할지

<ipython-input-19-eb0b6e709711>:32: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("TTS.mp3")
<ipython-input-19-eb0b6e709711>:50: UserWarning: PySoundFile failed. Trying audioread instead.
  y1, sr1 = librosa.load(file1, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio Similarity: 0.8988997
Final Presentation Score: 0.4494498372077942
